In [1]:
import os
os.chdir("/usr/src/app")
import pandas as pd
from os.path import join as jp
import cv2
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import re

In [2]:
from deepsetstats.paths import (
    PATH_VIDEOS,
    PATH_IMAGES_REFCOURT,
    PATH_MASTER_TENNIS_TV,
    PATH_MASTER_GS,
    PATH_TOURNAMENTS_NAMING, 
    PATH_REFERENCE_VIDEOS,
)
from deepsetstats.dataset.court_detection.utils import Utils

In [3]:
ut = Utils()
create_img_name = ut.create_img_name
full_parse_img_ref = ut.full_parse_img_ref
parse_img_ref = ut.parse_img_ref
bgr_to_rgb = ut.bgr_to_rgb
rgb_to_bgr = ut.rgb_to_bgr
plot_img_rgb = ut.plot_img_rgb
save_bgr_img = ut.save_bgr_img
get_random_frame = ut.get_random_frame
list_videos = ut.list_videos

# <font size=20> Tennis TV

# Get Downloaded Videos Id

In [4]:
# List all downloaded video_ids (tennistv + grand slams)
sv = list_videos(PATH_VIDEOS)
print("Videos downloaded:", len(sv))

Videos downloaded: 1198


# Load Master table

In [5]:
# Load all videos
df_tennistv = pd.read_parquet(PATH_MASTER_TENNIS_TV, engine="pyarrow")
df_gs = pd.read_parquet(PATH_MASTER_GS, engine="pyarrow")
df = pd.concat([df_tennistv, df_gs])

# load all tournaments
df_tt = pd.read_parquet(PATH_TOURNAMENTS_NAMING, engine="pyarrow")

# Ensure video is downloaded
df_data = df[df["video_id"].isin(sv)].copy()
df_data = pd.merge(df_data, df_tt[["tournament_name", "level"]].drop_duplicates(), on=["tournament_name"], how="left")

# Select one video Id per tournament

In [6]:
court_reference_videos = df_data.groupby(["tournament_name", "level"])["video_id"].first().reset_index()
court_reference_videos = court_reference_videos.sort_values("level")

In [7]:
cols = ["tournament_name", "tournament_id",]
court_reference_videos = pd.merge(
    court_reference_videos,
    df_tt[cols].drop_duplicates(),
    on = cols[0],
    how="left"
)

In [60]:
# Save the selected reference videos
# court_reference_videos.to_parquet(PATH_REFERENCE_VIDEOS, engine="pyarrow")

# Select reference court frame

Select a frame in which the court of that tournament can be clearly seen

In [9]:
court_reference_videos = pd.read_parquet(PATH_REFERENCE_VIDEOS, engine="pyarrow")

In [10]:
# Already existing tournaments in the reference
l_existing_ref = os.listdir(PATH_IMAGES_REFCOURT)

existing_tournament_ids_ref = set()
existing_video_ids_ref = set()
for name_imref in l_existing_ref:
    if name_imref.endswith(".png"):
        vid_id, tour_id = parse_img_ref(name_imref)
        existing_video_ids_ref.add(vid_id)
        existing_tournament_ids_ref.add(tour_id)

In [72]:
skipped_vids = set()

# Selector of the best reference court images
for i,row in court_reference_videos.iterrows():
    tournament_id = row["tournament_id"]
    video_id = row["video_id"]
    tour_name = row["tournament_name"]
    level = row["level"]

    if tournament_id in existing_tournament_ids_ref:
        print(f"Tournament: {tour_name}, already done, next one please!")
        continue
        
    if video_id in existing_video_ids_ref:
        print(f"Video: {video_id}, already done, next one please!")
        continue

    while True:
        print("==" * 30)
        print(f"Tournament: {tour_name}")
        print("==" * 30)
        img_bgr, frame_ok, frame_num = get_random_frame(video_id)
        user_input = input(f"Do you want to save? (y/n/s(skip): ")
        clear_output(wait=True)
        time.sleep(0.2)
        
        if user_input.lower() == "y":
            # Save the image and break the while loop to go to the next
            name_img = create_img_name(level, tournament_id, video_id, frame_num)
            path_img_ref = jp(PATH_IMAGES_REFCOURT, name_img)
            
            # TODO: uncomment to save
            #save_bgr_img(path=path_img_ref, img_bgr=img_bgr)
            print(f"Saved image: {name_img}")
            break
        elif user_input.lower() == "s":
            print(f"Skipping video")
            skipped_vids.add(video_id)
            break
        else:
            print("Not saving ! Generating a new image")
            time.sleep(0.5)
            clear_output(wait=True)
            continue

Tournament: Cincinnati, already done, next one please!
Tournament: Shanghai, already done, next one please!
Tournament: Paris, already done, next one please!
Tournament: Indian Wells, already done, next one please!
Tournament: Miami, already done, next one please!
Tournament: Monte Carlo, already done, next one please!
Tournament: Madrid, already done, next one please!
Tournament: Rome, already done, next one please!
Tournament: Toronto, already done, next one please!
Tournament: Adelaide, already done, next one please!
Tournament: Pune, already done, next one please!
Tournament: Auckland, already done, next one please!
Tournament: Adelaide, already done, next one please!
Tournament: Dallas, already done, next one please!
Tournament: Cordoba, already done, next one please!
Tournament: Montpellier, already done, next one please!
Tournament: Delray Beach, already done, next one please!
Tournament: Buenos Aires, already done, next one please!
Tournament: Doha, already done, next one pleas

# <font size=20> Summary Reference

In [11]:
l_imgs = os.listdir(PATH_IMAGES_REFCOURT)

In [15]:
l_tours = [full_parse_img_ref(img_x) for img_x in l_imgs]
df_ref_courts = pd.DataFrame(l_tours, columns=["filename", "level", "tournament_id", "video_id", "frame_num"])

In [16]:
cols = ["tournament_name", "tournament_id",]
df_ref_courts = pd.merge(
    df_ref_courts,
    df_tt[cols].drop_duplicates(),
    on = "tournament_id",
    how="left"
)

In [18]:
# df_ref_courts.to_parquet(PATH_REFERENCE_VIDEOS, engine="pyarrow")

In [19]:
df_ref_courts.head()

,filename,level,tournament_id,video_id,frame_num,tournament_name
0,ref___l1000___t10___vl6Dx9KzyRig___f2052.png,1000,10,l6Dx9KzyRig,2052,Cincinnati
1,ref___l1000___t11___vLhDFuGXXvGA___f5748.png,1000,11,LhDFuGXXvGA,5748,Shanghai
2,ref___l1000___t12___vrGT0FsEuc3U___f5199.png,1000,12,rGT0FsEuc3U,5199,Paris
3,ref___l1000___t4___vfnK4I6k9eHs___f29533.png,1000,4,fnK4I6k9eHs,29533,Indian Wells
4,ref___l1000___t5___vRdjN79TWRFQ___f8926.png,1000,5,RdjN79TWRFQ,8926,Miami
